In [ ]:
import cv2
import numpy as np
import os

In [ ]:
imgs = []

In [ ]:
filenames = sorted(os.listdir('imgs_test'), key=lambda x: int(x.split('.')[0]))

In [ ]:
# Load each image, convert it to a byte array, and append to the list
for filename in filenames:
    with open(f'imgs_test/{filename}', 'rb') as f:
        imgs.append(f.read())

In [ ]:
# Function to convert byte data to an image
def bytes_to_image(byte_data):
    np_arr = np.frombuffer(byte_data, np.uint8)
    image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return image


In [55]:
from ultralytics import YOLO

In [56]:
model = YOLO('../best.pt')

In [ ]:
def process_and_track(frames):
    for byte_data in frames:
        # Convert byte data to image
        print("vai se fuder")
        frame = bytes_to_image(byte_data)


        
        # Ensure the frame was converted successfully
        if frame is None:
            print('Failed to convert byte data to image')
            continue
        
        # Track objects in the frame
        results = model.track(source=frame)
        
        # Process the tracking results (e.g., draw bounding boxes)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
                label = result.names[int(box.cls[0])]
                confidence = box.conf[0].item()
                object_id = int(box.id[0])  # Assuming 'id' contains the unique ID
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                #cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        
        resized_frame = cv2.resize(frame, (500, 500))

        # Display the frame (optional, for visualization purposes)
        cv2.imshow('Tracked Frame', resized_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


In [ ]:
process_and_track(imgs)
cv2.destroyAllWindows()